
# NHANES Alcohol Columns — Clean Merge & Compute
**Goal:** Merge NHANES ALQ variables into your dataframe, compute **`drinking`** (drinks/day) and **`alcg2`** (alcohol category), and keep **`perE_alco`** unchanged unless you explicitly provide a vector to attach.

**What this notebook does:**
1. **Config**: set input/output file paths.
2. **Helpers**: dtype converters, URL mapping (fixes 404s), and CDC XPT fetchers.
3. **Load Base Data**: load your saved dataframe (`df_my_cov_aligned_short`).
4. **Fetch + Merge**: download ALQ (and DEMO if needed), merge on `SEQN`.
5. **Compute**: add `drinking` and `alcg2` with corrected logic (no false zeros).
6. **QC**: basic checks and previews.
7. **Save**: write the updated dataframe with alcohol columns to Parquet.


## 1) Config — paths and options

In [102]:

from pathlib import Path

# Input: your base dataframe (Parquet) that contains SEQN and existing covariates
IN_PARQUET = Path("/Users/dengshuyue/Desktop/SDOH/analysis/output/cov_addv2_99_23.parquet")

# Output: updated dataframe with added alcohol columns
OUT_PARQUET = Path("/Users/dengshuyue/Desktop/SDOH/analysis/output/cov_addv3_99_23.parquet") # with_alcohol

# Optional: provide a perE_alco vector here (must align with df index if used).
# Leave as None to keep existing perE_alco untouched.
perE_vec = None  # e.g., pd.Series([...], index=df_my_cov_aligned_short.index)


## 2) Helpers — dtype converters, cycle mapping, and CDC fetchers

In [103]:

import pandas as pd
import numpy as np
import io, requests
from typing import List, Optional

# ---- Dtype helpers ----
def to_float(s):
    return pd.to_numeric(s, errors="coerce")

def to_Int64(s):
    try:
        return pd.to_numeric(s, errors="coerce").astype("Int64")
    except Exception:
        return pd.to_numeric(s, errors="coerce")
    """
# ---- NHANES cycle → folder mapping (start year only; fixes 404s) ----
CYCLE_TO_FOLDER = {
    "":   "1999",  # 1999-2000 (no suffix)
    "_B": "2001",  # 2001-2002
    "_C": "2003",  # 2003-2004
    "_D": "2005",  # 2005-2006
    "_E": "2007",  # 2007-2008
    "_F": "2009",  # 2009-2010
    "_G": "2011",  # 2011-2012
    "_H": "2013",  # 2013-2014
    "_I": "2015",  # 2015-2016
    "_J": "2017",  # 2017-2018
}
SUFFIXES = ["", "_B","_C","_D","_E","_F","_G","_H","_I","_J"]

def cdc_xpt_url(folder: str, filebase_with_suffix: str) -> str:
    return f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/{folder}/DataFiles/{filebase_with_suffix}.xpt"

def fetch_xpt(filebase: str, suffix: str) -> pd.DataFrame:
    folder = CYCLE_TO_FOLDER[suffix]
    url = cdc_xpt_url(folder, f"{filebase}{suffix}")
    r = requests.get(url); r.raise_for_status()
    df = pd.read_sas(io.BytesIO(r.content), format="xport", encoding="latin1")
    df.columns = [c.upper() for c in df.columns]
    if "SEQN" in df.columns:
        df["SEQN"] = to_Int64(df["SEQN"])
    return df
    """

# ---- NHANES cycle → folder mapping ----
CYCLE_TO_FOLDER = {
    "":   "1999",
    "_B": "2001",
    "_C": "2003",
    "_D": "2005",
    "_E": "2007",
    "_F": "2009",
    "_G": "2011",
    "_H": "2013",
    "_I": "2015",
    "_J": "2017",  # 2017–2018 (normal J files, e.g. DEMO_J)
    "_P": "2017",  # 2017–Mar 2020 (special P_ALQ.xpt only)
    "_L": "2021",  # 2021–2023 from ALQ_L
}

SUFFIXES = ["", "_B","_C","_D","_E","_F","_G","_H","_I","_J","_P","_L"]

def fetch_xpt(filebase: str, suffix: str) -> pd.DataFrame:
    folder = CYCLE_TO_FOLDER[suffix]

    # Special naming for ALQ
    if filebase == "ALQ" and suffix == "_P":
        filebase_with_suffix = "P_ALQ"   # pre-pandemic 2017–Mar 2020
    elif filebase == "ALQ" and suffix == "_L":
        filebase_with_suffix = "ALQ_L"   # 2021–2023
    else:
        filebase_with_suffix = f"{filebase}{suffix}"

    url = f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/{folder}/DataFiles/{filebase_with_suffix}.xpt"
    r = requests.get(url); r.raise_for_status()
    df = pd.read_sas(io.BytesIO(r.content), format="xport", encoding="latin1")
    df.columns = [c.upper() for c in df.columns]
    if "SEQN" in df.columns:
        df["SEQN"] = to_Int64(df["SEQN"])
    return df


def fetch_stack(filebase: str, suffixes: List[str]) -> pd.DataFrame:
    parts = []
    for s in suffixes:
        try:
            parts.append(fetch_xpt(filebase, s))
        except Exception as e:
            print(f"Warning: {filebase}{s} skipped ({e})")
    return pd.concat(parts, axis=0, ignore_index=True) if parts else pd.DataFrame()

# ---- ALQ & DEMO fetchers ----
ALQ_COLS = ["SEQN","ALQ100","ALQ101","ALQ110","ALQ111","ALQ120Q","ALQ120U","ALQ121","ALQ130"]
DEMO_COLS = ["SEQN","RIAGENDR"]

def fetch_alq_all() -> pd.DataFrame:
    alq = fetch_stack("ALQ", SUFFIXES)
    if alq.empty: return alq
    keep = [c for c in ALQ_COLS if c in alq.columns]
    return alq[keep].copy()

def fetch_demo_sex_all() -> pd.DataFrame:
    demo = fetch_stack("DEMO", SUFFIXES)
    if demo.empty: return demo
    keep = [c for c in DEMO_COLS if c in demo.columns]
    return demo[keep].copy()


## 3) Load base dataframe (`df_my_cov_aligned_short`)

In [104]:

df_my_cov_aligned_short = pd.read_parquet(IN_PARQUET)
df_my_cov_aligned_short["SEQN"] = to_Int64(df_my_cov_aligned_short["SEQN"])
print("Loaded df_my_cov_aligned_short:", df_my_cov_aligned_short.shape)


Loaded df_my_cov_aligned_short: (128809, 60)


## 4) Fetch ALQ (and DEMO if needed), then merge by `SEQN`

In [105]:

alq = fetch_alq_all()
print("ALQ shape:", alq.shape)
if not alq.empty:
    alq["SEQN"] = to_Int64(alq["SEQN"])
    df_my_cov_aligned_short = df_my_cov_aligned_short.merge(
        alq, on="SEQN", how="left", suffixes=("", "_ALQ")
    )

needs_sex = ("RIAGENDR" not in df_my_cov_aligned_short.columns or
             df_my_cov_aligned_short["RIAGENDR"].isna().all())
if needs_sex:
    demo = fetch_demo_sex_all()
    print("DEMO shape:", demo.shape)
    if not demo.empty:
        demo["SEQN"] = to_Int64(demo["SEQN"])
        df_my_cov_aligned_short = df_my_cov_aligned_short.merge(
            demo, on="SEQN", how="left", suffixes=("", "_DEMO")
        )

print("After merge:", df_my_cov_aligned_short.shape)


ALQ shape: (68861, 9)
DEMO shape: (113249, 2)
After merge: (128809, 69)


## 5) Compute `drinking` and `alcg2` (Lu SAS logic, fixed)

In [106]:

from typing import Optional

def add_alcohol_columns(df: pd.DataFrame, perE_series: Optional[pd.Series] = None) -> pd.DataFrame:
    # Ensure ALQ fields exist
    for c in ["ALQ100","ALQ101","ALQ110","ALQ111","ALQ120Q","ALQ120U","ALQ121","ALQ130"]:
        if c not in df.columns: df[c] = np.nan
    if "RIAGENDR" not in df.columns: df["RIAGENDR"] = np.nan

    w = df.rename(columns=str.lower).copy()

    # Normalize and handle special missings
    w["alq130"]  = pd.to_numeric(w["alq130"],  errors="coerce").replace({777: np.nan, 999: np.nan})
    w["alq120q"] = pd.to_numeric(w["alq120q"], errors="coerce").replace({777: np.nan, 999: np.nan})
    for c in ["alq100","alq101","alq110","alq111","alq121","alq120u","riagendr"]:
        w[c] = pd.to_numeric(w[c], errors="coerce")

    # Only explicit zeros / flags mean no drinking
    no_drink_flag = (
        (w["alq101"] == 2) |
        (w["alq100"] == 2) |
        (w["alq110"] == 2) |
        (w["alq120q"].eq(0)) |
        (w["alq121"].eq(0))
    )
    w.loc[no_drink_flag, "alq130"] = 0.0

    # Tiny nonzero if reported drinking but alq130 missing
    w.loc[(w["alq100"] == 1) & (w["alq130"].isna()), "alq130"] = 0.03

    # Compute drinks/day
    drinking = pd.Series(np.nan, index=w.index, dtype="float64")
    m = (w["alq120u"] == 1); drinking = drinking.where(~m, (w["alq120q"]/7.0)   * w["alq130"])
    m = (w["alq120u"] == 2); drinking = drinking.where(~m, (w["alq120q"]/30.0)  * w["alq130"])
    m = (w["alq120u"] == 3); drinking = drinking.where(~m, (w["alq120q"]/365.0) * w["alq130"])

    mapping = {1:1.0, 2:6/7, 3:3.5/7, 4:2/7, 5:1/7, 6:1/12.5, 7:1/30, 8:9/365, 9:4.5/365, 10:1.5/365}
    mult = w["alq121"].map(mapping)
    drinking = np.where(mult.notna(), w["alq130"]*mult, drinking)

    # Final zeroing only when explicit flags or alq130 == 0
    no_drink_final = (
        (w["alq110"] == 2) | (w["alq111"] == 2) | (w["alq101"] == 2) |
        (w["alq100"] == 2) | (w["alq121"].eq(0)) | (w["alq130"].eq(0))
    )
    drinking = np.where(no_drink_final, 0.0, drinking)
    df["drinking"] = to_float(drinking)

    # alcg2 categories
    alcg2 = pd.Series(pd.NA, index=df.index)
    alcg2 = alcg2.mask((df["drinking"].ge(0)) & (df["drinking"].lt(0.03)), 1)                # none/rare
    alcg2 = alcg2.mask((df["RIAGENDR"]==1) & (df["drinking"].ge(0.03)) & (df["drinking"].lt(2)), 2)  # male light/mod
    alcg2 = alcg2.mask((df["RIAGENDR"]==2) & (df["drinking"].ge(0.03)) & (df["drinking"].lt(1)), 2)  # female light/mod
    alcg2 = alcg2.mask((df["RIAGENDR"]==1) & (df["drinking"].ge(2)), 3)                       # male heavy
    alcg2 = alcg2.mask((df["RIAGENDR"]==2) & (df["drinking"].ge(1)), 3)                       # female heavy
    try:
        df["alcg2"] = alcg2.astype("Int64")
    except Exception:
        df["alcg2"] = to_Int64(alcg2)

    # perE_alco: keep as-is unless a series is provided
    if perE_vec is not None:
        s = to_float(perE_vec)
        if not s.index.equals(df.index):
            s = s.reindex(df.index)
        df["perE_alco"] = s

    # QC prints
    nn = w[["alq100","alq101","alq110","alq111","alq120q","alq120u","alq121","alq130"]].notna().sum()
    print("Non-missing ALQ counts:\n", nn.to_string())
    print("Nonzero alq130:", (w["alq130"]>0).sum(), "| Nonzero drinking:", (df["drinking"]>0).sum())

    return df

df_my_cov_aligned_short = add_alcohol_columns(df_my_cov_aligned_short, perE_series=perE_vec)
print("After compute:", df_my_cov_aligned_short.shape)


Non-missing ALQ counts:
 alq100      4161
alq101     34838
alq110     13357
alq111     18981
alq120q    36773
alq120u    28412
alq121     16970
alq130     52089
Nonzero alq130: 39524 | Nonzero drinking: 39030
After compute: (128809, 71)


## 6) QC & Preview

In [107]:

print(df_my_cov_aligned_short[["drinking","alcg2"]].dtypes)
if "alcg2" in df_my_cov_aligned_short:
    print("\n`alcg2` counts:")
    print(df_my_cov_aligned_short["alcg2"].value_counts(dropna=False).head(12))

nz = df_my_cov_aligned_short.loc[df_my_cov_aligned_short["drinking"]>0,
                                 ["SEQN","RIAGENDR","ALQ120Q","ALQ120U","ALQ121","ALQ130","drinking","alcg2"]]
print("\nRows with nonzero drinking (head):")
print(nz.head(12))


drinking    float64
alcg2         Int64
dtype: object

`alcg2` counts:
alcg2
<NA>    79484
1       22388
2       22325
3        4612
Name: count, dtype: Int64

Rows with nonzero drinking (head):
    SEQN  RIAGENDR  ALQ120Q  ALQ120U  ALQ121  ALQ130  drinking  alcg2
1      2       1.0     24.0      3.0     NaN     1.0  0.065753      2
4      5       1.0      4.0      1.0     NaN     3.0  1.714286      2
9     10       1.0      6.0      3.0     NaN     1.0  0.016438      1
11    12       1.0      2.0      2.0     NaN     3.0  0.200000      2
12    13       1.0      1.0      1.0     NaN     2.0  0.285714      2
13    14       1.0      1.0      1.0     NaN     1.0  0.142857      2
14    15       2.0      4.0      1.0     NaN     2.0  1.142857      3
19    20       2.0      3.0      1.0     NaN     3.0  1.285714      3
23    24       2.0      4.0      1.0     NaN     4.0  2.285714      3
24    25       2.0      4.0      1.0     NaN     2.0  1.142857      3
32    33       1.0      4.0      1.

#### sanity check 


In [108]:
SEQNS_CHECK = [31131, 31132, 31144]  # add as many as you want

for sid in SEQNS_CHECK:
    row = df_my_cov_aligned_short.loc[df_my_cov_aligned_short["SEQN"] == sid]
    if row.empty:
        print(f"\n⚠️ SEQN {sid} not found in dataframe.")
        continue

    r = row.squeeze()
    print("\n" + "="*40)
    print(f"SEQN {sid}")

    # Show inputs + outputs
    cols_show = ["SEQN","RIAGENDR","ALQ100","ALQ101","ALQ110","ALQ111",
                 "ALQ120Q","ALQ120U","ALQ121","ALQ130","drinking","alcg2","perE_alco"]
    print(row[[c for c in cols_show if c in row.columns]])


SEQN 31131
        SEQN  RIAGENDR  ALQ100  ALQ101  ALQ110  ALQ111       ALQ120Q  ALQ120U  \
31130  31131       2.0     NaN     2.0     1.0     NaN  5.397605e-79      NaN   

       ALQ121  ALQ130  drinking  alcg2  
31130     NaN     NaN       0.0      1  

SEQN 31132
        SEQN  RIAGENDR  ALQ100  ALQ101  ALQ110  ALQ111  ALQ120Q  ALQ120U  \
31131  31132       1.0     NaN     1.0     NaN     NaN      4.0      1.0   

       ALQ121  ALQ130  drinking  alcg2  
31131     NaN     1.0  0.571429      2  

SEQN 31144
        SEQN  RIAGENDR  ALQ100  ALQ101  ALQ110  ALQ111  ALQ120Q  ALQ120U  \
31143  31144       1.0     NaN     1.0     NaN     NaN      2.0      2.0   

       ALQ121  ALQ130  drinking  alcg2  
31143     NaN     2.0  0.133333      2  


## 7) Add perE_alco

In [109]:
import pandas as pd
import numpy as np
import io, requests

# ---- tiny helpers (safe even if you already defined similar ones) ----
def _to_Int64(s):
    try:
        return pd.to_numeric(s, errors="coerce").astype("Int64")
    except Exception:
        return pd.to_numeric(s, errors="coerce")

CYCLE_TO_FOLDER = {"":"1999","_B":"2001","_C":"2003","_D":"2005","_E":"2007",
                   "_F":"2009","_G":"2011","_H":"2013","_I":"2015","_J":"2017"}

def _cdc_url(folder, filebase_with_suffix):
    return f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/{folder}/DataFiles/{filebase_with_suffix}.xpt"

def _fetch_xpt(filebase, suffix):
    folder = CYCLE_TO_FOLDER[suffix]
    url = _cdc_url(folder, f"{filebase}{suffix}")
    r = requests.get(url); r.raise_for_status()
    df = pd.read_sas(io.BytesIO(r.content), format="xport", encoding="latin1")
    df.columns = [c.upper() for c in df.columns]
    if "SEQN" in df.columns:
        df["SEQN"] = _to_Int64(df["SEQN"])
    return df

def _fetch_stack(filebase, suffixes):
    parts = []
    for s in suffixes:
        try:
            parts.append(_fetch_xpt(filebase, s))
        except Exception as e:
            print(f"Warning: {filebase}{s} skipped ({e})")
    return pd.concat(parts, ignore_index=True) if parts else pd.DataFrame()

def add_perE_alco(df, eps=1e-6):  # <<< add EPS threshold here
    """Create/attach perE_alco (% energy from alcohol) from NHANES dietary totals."""
    # 1999–2002: DRXTOT (day 1 only)
    drx = _fetch_stack("DRXTOT", ["", "_B"])
    if not drx.empty:
        for c in ("DRXTKCAL","DRXTALCO"):
            if c in drx:
                drx[c] = pd.to_numeric(drx[c], errors="coerce")
                # <<< clean denormals / negatives
                drx.loc[drx[c].abs() < eps, c] = np.nan
                drx.loc[drx[c] < 0, c] = np.nan
        drx_std = drx.rename(columns={"DRXTKCAL":"kcal", "DRXTALCO":"alco_g"})[["SEQN","kcal","alco_g"]]
    else:
        drx_std = pd.DataFrame(columns=["SEQN","kcal","alco_g"])

    # 2003–2018 day 1 & day 2
    suffixes_03_18 = ["_C","_D","_E","_F","_G","_H","_I","_J"]

    dr1 = _fetch_stack("DR1TOT", suffixes_03_18)
    if not dr1.empty:
        # prefer DR1TALCOG if present, else DR1TALCO
        alc1 = "DR1TALCOG" if "DR1TALCOG" in dr1.columns else "DR1TALCO"
        for c in ("DR1TKCAL", alc1):
            if c in dr1:
                dr1[c] = pd.to_numeric(dr1[c], errors="coerce")
                # <<< clean denormals / negatives
                dr1.loc[dr1[c].abs() < eps, c] = np.nan
                dr1.loc[dr1[c] < 0, c] = np.nan
        d1_std = dr1.rename(columns={"DR1TKCAL":"kcal", alc1:"alco_g"})[["SEQN","kcal","alco_g"]]
    else:
        d1_std = pd.DataFrame(columns=["SEQN","kcal","alco_g"])

    dr2 = _fetch_stack("DR2TOT", suffixes_03_18)
    if not dr2.empty:
        # prefer DR2TALCOG if present, else DR2TALCO
        alc2 = "DR2TALCOG" if "DR2TALCOG" in dr2.columns else "DR2TALCO"
        for c in ("DR2TKCAL", alc2):
            if c in dr2:
                dr2[c] = pd.to_numeric(dr2[c], errors="coerce")
                # <<< clean denormals / negatives
                dr2.loc[dr2[c].abs() < eps, c] = np.nan
                dr2.loc[dr2[c] < 0, c] = np.nan
        d2_std = dr2.rename(columns={"DR2TKCAL":"kcal", alc2:"alco_g"})[["SEQN","kcal","alco_g"]]
    else:
        d2_std = pd.DataFrame(columns=["SEQN","kcal","alco_g"])

    # Combine all available days
    all_days = pd.concat([drx_std, d1_std, d2_std], ignore_index=True)
    if all_days.empty:
        print("No dietary totals fetched; creating empty perE_alco.")
        if "perE_alco" not in df.columns:
            df["perE_alco"] = pd.Series([pd.NA]*len(df))
        return df

    # Alcohol kcal = 7 kcal/g
    all_days["alc_kcal"] = all_days["alco_g"].fillna(0) * 7.0

    agg = (all_days.groupby("SEQN", as_index=False)
           .agg(total_kcal=("kcal","sum"),
                total_alc_kcal=("alc_kcal","sum")))

    # <<< guard denominators & tiny totals
    agg.loc[agg["total_kcal"].abs() < eps, "total_kcal"] = np.nan
    agg.loc[agg["total_alc_kcal"].abs() < eps, "total_alc_kcal"] = 0.0

    agg["perE_alco"] = np.where(
        agg["total_kcal"].notna() & (agg["total_kcal"] > 0),
        100.0 * agg["total_alc_kcal"] / agg["total_kcal"],
        np.nan
    )

    # <<< snap any residual tiny positives to exact 0 (optional but nice)
    tiny = agg["perE_alco"].abs() < eps
    agg.loc[tiny, "perE_alco"] = 0.0

    # Merge back (always ensure column exists)
    df["SEQN"] = _to_Int64(df["SEQN"])
    df = df.merge(agg[["SEQN","perE_alco"]], on="SEQN", how="left")
    if "perE_alco" not in df.columns:
        df["perE_alco"] = pd.NA

    # Sanity prints
    nn = df["perE_alco"].notna().sum()
    nz = (df["perE_alco"] > 0).sum(skipna=True)
    print(f"perE_alco non-missing: {nn} | >0: {nz}")
    print(df.loc[df["perE_alco"]>0, ["SEQN","perE_alco"]].head(10))

    return df


In [110]:
df_my_cov_aligned_short = add_perE_alco(df_my_cov_aligned_short)

# Verify the column exists to avoid future KeyErrors:
print("perE_alco in columns?", "perE_alco" in df_my_cov_aligned_short.columns)
print(df_my_cov_aligned_short[["SEQN","perE_alco"]].head())


perE_alco non-missing: 88019 | >0: 15367
    SEQN  perE_alco
4      5   9.101101
10    11   0.018316
14    15   3.577341
23    24  24.034478
32    33   4.219476
34    35   0.004085
41    42   0.022075
44    45   0.044028
55    56   6.848446
56    57  20.755760
perE_alco in columns? True
   SEQN  perE_alco
0     1   0.000000
1     2   0.000000
2     3   0.000000
3     4   0.000000
4     5   9.101101


In [111]:
SEQNS_CHECK = [31131, 31132, 31144, 31149, 31150, 31158]  # add as many as you want

for sid in SEQNS_CHECK:
    row = df_my_cov_aligned_short.loc[df_my_cov_aligned_short["SEQN"] == sid]
    if row.empty:
        print(f"\n⚠️ SEQN {sid} not found in dataframe.")
        continue

    r = row.squeeze()
    print("\n" + "="*40)
    print(f"SEQN {sid}")

    # Show inputs + outputs
    cols_show = ["SEQN","RIAGENDR","ALQ100","ALQ101","ALQ110","ALQ111",
                 "ALQ120Q","ALQ120U","ALQ121","ALQ130","drinking","alcg2","perE_alco"]
    print(row[[c for c in cols_show if c in row.columns]])


SEQN 31131
        SEQN  RIAGENDR  ALQ100  ALQ101  ALQ110  ALQ111       ALQ120Q  ALQ120U  \
31130  31131       2.0     NaN     2.0     1.0     NaN  5.397605e-79      NaN   

       ALQ121  ALQ130  drinking  alcg2  perE_alco  
31130     NaN     NaN       0.0      1        0.0  

SEQN 31132
        SEQN  RIAGENDR  ALQ100  ALQ101  ALQ110  ALQ111  ALQ120Q  ALQ120U  \
31131  31132       1.0     NaN     1.0     NaN     NaN      4.0      1.0   

       ALQ121  ALQ130  drinking  alcg2  perE_alco  
31131     NaN     1.0  0.571429      2  12.008854  

SEQN 31144
        SEQN  RIAGENDR  ALQ100  ALQ101  ALQ110  ALQ111  ALQ120Q  ALQ120U  \
31143  31144       1.0     NaN     1.0     NaN     NaN      2.0      2.0   

       ALQ121  ALQ130  drinking  alcg2  perE_alco  
31143     NaN     2.0  0.133333      2   6.543909  

SEQN 31149
        SEQN  RIAGENDR  ALQ100  ALQ101  ALQ110  ALQ111  ALQ120Q  ALQ120U  \
31148  31149       2.0     NaN     2.0     2.0     NaN      NaN      NaN   

       ALQ121  ALQ

## 8) Save updated dataframe

In [112]:

df_my_cov_aligned_short.to_parquet(OUT_PARQUET, index=False)
print("✓ Saved:", OUT_PARQUET)


✓ Saved: /Users/dengshuyue/Desktop/SDOH/analysis/output/cov_addv3_99_23.parquet


## 9) check missingness again 

In [113]:
import pandas as pd
from pathlib import Path

p = Path("/Users/dengshuyue/Desktop/SDOH/analysis/output/cov_addv3_99_23.parquet")
df_my_cov_aligned_short = pd.read_parquet(p)  # uses pyarrow/fastparquet if available
print(df_my_cov_aligned_short.shape)
df_my_cov_aligned_short.head()


(128809, 72)


,SEQN,SDDSRVYR,sdmvpsu,sdmvstra,RIDAGEYR,SEX,RACE,household_size,EDU,pir,...,ALQ110,ALQ111,ALQ120Q,ALQ120U,ALQ121,ALQ130,RIAGENDR,drinking,alcg2,perE_alco
0,1,1.0,1,5,2.0,F,4.0,3,<NA>,0.86,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,<NA>,0.000000
1,2,1.0,3,1,77.0,M,3.0,1,5,5.00,...,NaN,NaN,24.0,3.0,NaN,1.0,1.0,0.065753,2,0.000000
2,3,1.0,2,7,10.0,F,3.0,4,3,1.47,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,<NA>,0.000000
3,4,1.0,1,2,1.0,M,4.0,7,<NA>,0.57,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,<NA>,0.000000
4,5,1.0,2,8,49.0,M,3.0,3,5,5.00,...,NaN,NaN,4.0,1.0,NaN,3.0,1.0,1.714286,2,9.101101


#### remove uncessary alq columns

In [114]:
# Keep only these 3 alcohol fields
keep_alc = {"drinking", "alcg2", "perE_alco"}

cols = df_my_cov_aligned_short.columns

# Flag alcohol-related columns by common patterns/prefixes (case-insensitive)
alc_cols = [c for c in cols if (
    c.upper().startswith(("ALQ", "ALC", "DRINK")) or  # catches ALQ120U/Q, ALC*, DRINKS_PER_DAY, drinking
    "ALCOHOL" in c.upper()
)]

# Drop all flagged alcohol columns except the three keepers
to_drop = [c for c in alc_cols if c.lower() not in keep_alc]

print("Dropping:", sorted(to_drop))
df_my_cov_aligned_short = df_my_cov_aligned_short.drop(columns=to_drop, errors="ignore")


Dropping: ['ALCOHOL_CAT', 'ALQ100', 'ALQ101', 'ALQ110', 'ALQ111', 'ALQ120Q', 'ALQ120U', 'ALQ121', 'ALQ130', 'DRINKS_PER_DAY']


In [115]:
df_my_cov_aligned_short.columns

Index(['SEQN', 'SDDSRVYR', 'sdmvpsu', 'sdmvstra', 'RIDAGEYR', 'SEX', 'RACE',
       'household_size', 'EDU', 'pir', 'SMK_AVG', 'SMK', 'ALCG2', 'met_hr',
       'SMK_STATUS', 'CIGS_PER_DAY', 'PACK_YEARS', 'FORMER_SMOKER', 'bmi_cat',
       'DIABE', 'HYPERTEN', 'chol_rx', 'CVD', 'cancer', 'probable_depression',
       'ahei_total', 'unemployment2', 'sdoh_access', 'ins', 'HOQ065',
       'marriage', 'SNAP', 'FS', 'WTINT2YR', 'WTMEC2YR', 'WTSAF2YR',
       'WTINT4YR', 'WTMEC4YR', 'WTINTPRP', 'WTMECPRP', 'WTSAFPRP', 'wt_int',
       'wt_mec', 'wt_fasting', 'wt_phlebotomy', 'WTPH2YR', 'marriage_prev',
       'marriage_label', 'marriage3', 'SNAP_src', 'SNAP_bin', 'SNAP_src_rank',
       'SNAP_indiv_only', 'SNAP_indiv_plus_singleton', 'bmi',
       'HOQ065_structural_missing', 'household_size_structural_missing',
       'chol_rx_structural_missing', 'RIAGENDR', 'drinking', 'alcg2',
       'perE_alco'],
      dtype='object')

In [117]:
import pandas as pd

# Alcohol vars to audit (keep only those that exist)
alc_core_all = ["drinking", "alcg2", "perE_alco"]
alc_core = [c for c in alc_core_all if c in df_my_cov_aligned_short.columns]
if not alc_core:
    raise ValueError("None of the alcohol variables are present.")

# Full summary (n, n_miss, pct_miss, pct_nonmiss) by cycle
miss_summary = (
    df_my_cov_aligned_short
    .groupby("SDDSRVYR")[alc_core]
    .apply(lambda g: pd.DataFrame({
        "n": len(g),
        "n_miss": g.isna().sum(),
        "pct_miss": (g.isna().mean() * 100).round(2),
        "pct_nonmiss": ((1 - g.isna().mean()) * 100).round(2)
    }).T)
)

# Show nicely:
pd.set_option("display.max_rows", 200)
# print("Alcohol missingness by cycle:")
# print(miss_summary)

# If you want a lighter table with only % missing:
pct_missing = (
    df_my_cov_aligned_short
    .groupby("SDDSRVYR")[alc_core]
    .apply(lambda g: (g.isna().mean() * 100).round(2))
    .rename(columns=lambda c: f"{c}_pct_miss")
)
print("\n% missing by cycle (lighter):")
print(pct_missing)

# Optional: save to CSVs
# miss_summary.to_csv("alcohol_missingness_full.csv")
# pct_missing.to_csv("alcohol_pct_missing.csv")



% missing by cycle (lighter):
          drinking_pct_miss  alcg2_pct_miss  perE_alco_pct_miss
SDDSRVYR                                                       
1.0                   62.87           62.87               12.48
2.0                   66.48           66.48               12.13
3.0                   62.88           62.88               12.06
4.0                   62.67           62.67               11.33
5.0                   55.41           55.41               10.12
6.0                   55.38           55.38                8.59
7.0                   55.06           55.06               13.91
8.0                   52.17           52.17               16.12
9.0                   52.76           52.76               16.38
10.0                  56.01           56.01               19.01
12.0                  61.38           61.38              100.00
66.0                  56.83           84.34              100.00


In [118]:
df_my_cov_aligned_short[["chol_rx", "met_hr"]]

,chol_rx,met_hr
0,0,NaN
1,0,60.0
2,0,NaN
3,0,NaN
4,0,1920.0
...,...,...
128804,<NA>,NaN
128805,1,180.0
128806,0,180.0
128807,0,840.0
